<i> In previous part we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.


The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code.

<i>1. Importing libraries

In [3]:
import numpy as np
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

Solving environment: ...working... done

# All requested packages already installed.



<i>I will use the Geocoder Python package to get the latitude and the longitude coordinates of each neighborhood: https://geocoder.readthedocs.io/index.html.

In [4]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: ...working... done

# All requested packages already installed.



<i>Getting list of zip codes

In [6]:
data = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050').text

In [7]:
soup = BeautifulSoup(data, 'html.parser')

In [8]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [9]:

soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [10]:

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

<i>Creating Dataframe

In [11]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [12]:
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))

In [13]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

In [14]:

column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)

In [15]:
toronto_df_grouped.shape

(103, 3)

<i>Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [16]:

coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:

toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<i>Task2: Creating dataframe with coordinates

In [19]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


In [20]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Exploring and clustering the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:
<ul>
<li>to add enough Markdown cells to explain what you decided to do and to report any observations you make.
<li>to generate maps to visualize your neighborhoods and how they cluster together.

<i>I will use folium library for renderiring map of Toronto

In [39]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: ...working... done

# All requested packages already installed.



In [21]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [22]:
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

<i>creating a new DataFrame with only boroughs that contain the word Toronto

In [23]:

toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


<i>add markers to map

In [24]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [26]:
CLIENT_ID = 'IC4IOXYCFF1LS00NIPEPXITNLFSLCSG1AXBT4HCHVOQPKOJK' # your Foursquare ID
CLIENT_SECRET = 'FBBIR2WXU4MLRNEQKTU3JPYMI0QLGNZPWANVFIXG4KC5P2NC'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IC4IOXYCFF1LS00NIPEPXITNLFSLCSG1AXBT4HCHVOQPKOJK
CLIENT_SECRET:FBBIR2WXU4MLRNEQKTU3JPYMI0QLGNZPWANVFIXG4KC5P2NC


In [28]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], 
                                                  toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=IC4IOXYCFF1LS00NIPEPXITNLFSLCSG1AXBT4HCHVOQPKOJK&client_secret=FBBIR2WXU4MLRNEQKTU3JPYMI0QLGNZPWANVFIXG4KC5P2NC&v=20180605 \
     &ll=43.653963,-79.387207&radius=500&limit=100".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [29]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2886, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Japango,43.655268,-79.385165,Sushi Restaurant
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Poke Guys,43.654895,-79.385052,Poke Place
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Rolltation,43.654918,-79.387424,Japanese Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant


In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(2886, 57)


,PostalCode,Borough,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bar,Breakfast Spot,...,Seafood Restaurant,Smoke Shop,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Toy / Game Store,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 57)


,PostalCode,Borough,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bar,Breakfast Spot,...,Seafood Restaurant,Smoke Shop,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Toy / Game Store,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0.013514,0.040541,0.013514,0.013514,0.013514,0.027027,0.040541,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.027027,0.013514,0.013514,0.013514,0.027027
1,M4K,East Toronto,"The Danforth West, Riverdale",0.013514,0.040541,0.013514,0.013514,0.013514,0.027027,0.040541,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.027027,0.013514,0.013514,0.013514,0.027027
2,M4L,East Toronto,"The Beaches West, India Bazaar",0.013514,0.040541,0.013514,0.013514,0.013514,0.027027,0.040541,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.027027,0.013514,0.013514,0.013514,0.027027
3,M4M,East Toronto,Studio District,0.013514,0.040541,0.013514,0.013514,0.013514,0.027027,0.040541,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.027027,0.013514,0.013514,0.013514,0.027027
4,M4N,Central Toronto,Lawrence Park,0.013514,0.040541,0.013514,0.013514,0.013514,0.027027,0.040541,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.027027,0.013514,0.013514,0.013514,0.027027
5,M4P,Central Toronto,Davisville North,0.013514,0.040541,0.013514,0.013514,0.013514,0.027027,0.040541,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.027027,0.013514,0.013514,0.013514,0.027027
6,M4R,Central Toronto,North Toronto West,0.013514,0.040541,0.013514,0.013514,0.013514,0.027027,0.040541,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.027027,0.013514,0.013514,0.013514,0.027027
7,M4S,Central Toronto,Davisville,0.013514,0.040541,0.013514,0.013514,0.013514,0.027027,0.040541,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.027027,0.013514,0.013514,0.013514,0.027027
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.013514,0.040541,0.013514,0.013514,0.013514,0.027027,0.040541,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.027027,0.013514,0.013514,0.013514,0.027027
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.013514,0.040541,0.013514,0.013514,0.013514,0.027027,0.040541,...,0.013514,0.013514,0.013514,0.027027,0.013514,0.027027,0.013514,0.013514,0.013514,0.027027


In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
3,M4M,East Toronto,Studio District,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
4,M4N,Central Toronto,Lawrence Park,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
5,M4P,Central Toronto,Davisville North,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
6,M4R,Central Toronto,North Toronto West,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
7,M4S,Central Toronto,Davisville,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant


In [33]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

C:\Users\jrslw\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:969: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [34]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant


In [35]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
23,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
25,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
26,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
27,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
20,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
28,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant


In [36]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
21,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
22,Central Toronto,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
23,Central Toronto,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
24,Central Toronto,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
25,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
26,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
27,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
20,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
28,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bar,Hotel,Chinese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
